In [3]:
!pip install openai-whisper gradio transformers scikit-learn torch
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
import whisper
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import gradio as gr


try:
    whisper_model = whisper.load_model("base")
except Exception as e:
    print("Error loading Whisper model:", e)


try:
    sentiment_pipeline = pipeline("sentiment-analysis")
except Exception as e:
    print("Error loading Sentiment model:", e)

# 🔹 Transcription
def transcribe_audio(audio_path):
    try:
        result = whisper_model.transcribe(audio_path)
        return result['text']
    except Exception as e:
        return f"Transcription Error: {str(e)}"

# 🔹 Sentiment Analysis
def analyze_sentiment(text):
    try:
        result = sentiment_pipeline(text)[0]
        return f"{result['label']} ({round(result['score'], 2)})"
    except Exception as e:
        return f"Sentiment Error: {str(e)}"

# 🔹 Keyword Extraction
def extract_keywords(text, top_n=5):
    try:
        vectorizer = TfidfVectorizer(stop_words='english')
        X = vectorizer.fit_transform([text])
        scores = zip(vectorizer.get_feature_names_out(), X.toarray()[0])
        sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
        return [word for word, score in sorted_scores[:top_n]]
    except Exception as e:
        return [f"Keyword Error: {str(e)}"]

# 🔹 Agent Reasoning
def agent_decision(sentiment, keywords):
    try:
        decisions = []
        if "NEGATIVE" in sentiment.upper():
            decisions.append("⚠️ Tone sounds negative, might need more confidence.")
        else:
            decisions.append("✅ Tone is positive, candidate seems confident.")

        if not keywords or "Keyword Error" in keywords[0]:
            decisions.append("❓ Could not identify relevant skills.")
        else:
            decisions.append(f"🔍 Skills detected: {', '.join(keywords)}")

        decisions.append("👉 Recommend polite closing & follow-up call by HR.")
        return decisions
    except Exception as e:
        return [f"Decision Error: {str(e)}"]

# 🔹 Final Wrapper Function
def process_audio(audio):
    try:
        transcription = transcribe_audio(audio)
        if transcription.startswith("Transcription Error"):
            return transcription, "Error", "Error", "Error"

        sentiment = analyze_sentiment(transcription)
        if sentiment.startswith("Sentiment Error"):
            return transcription, sentiment, "Error", "Error"

        keywords = extract_keywords(transcription)
        if "Keyword Error" in keywords[0]:
            return transcription, sentiment, ', '.join(keywords), "Error"

        decisions = agent_decision(sentiment, keywords)

        return transcription, sentiment, ', '.join(keywords), '\n'.join(decisions)
    except Exception as e:
        return f"Unexpected Error: {str(e)}", "Error", "Error", "Error"

# 🔹 Gradio Interface
interface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="Upload Candidate Voice (WAV/MP3)"),
    outputs=[
        gr.Textbox(label="📝 Transcription"),
        gr.Textbox(label="🎯 Sentiment"),
        gr.Textbox(label="🧠 Keywords"),
        gr.Textbox(label="🤖 Agent Decisions")
    ],
    title="🎙️ Voice AI Agent for Candidate Screening",
    description="Upload a candidate's voice response. The AI will transcribe, analyze tone, extract skills, and suggest next actions.",
    theme="default"
)

# Run
if __name__ == "__main__":
    interface.launch()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ce64b2fe222dabe68.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
